In [75]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes 

# map rendering library
import folium 
print('Libraries imported.')

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Libraries imported.


Download the neighborhood names from wikipedia

In [76]:
Rurl= 'https://en.wikipedia.org/wiki/Riyadh'
riyadh_data=requests.get(Rurl).text
print("the data is downloaded")

the data is downloaded


In [120]:
#riyadh_data

Use BeautifulSoup to extract the neighborhood names

In [121]:

from bs4 import BeautifulSoup
soup = BeautifulSoup(riyadh_data, 'html.parser')
#soup

In [122]:
#print(soup.prettify())

In [123]:
#represents the document as a nested data structure
#print(soup.prettify())

#find the div with class = div-col columns column-widthe
div = soup.find('div',{'class':'div-col columns column-width'})

#find all <li> tags
li_rows= div.find_all('li')

#li_rows

In [124]:
#<li>  </li>
data = []
for row in li_rows:
    if '\n' in row:
        continue
    else:
        data.append(row.text.strip())
df = pd.DataFrame(data, columns=['Neighborhood'])
df

,Neighborhood
0,Al-Deerah (old Riyadh)
1,Mi'kal
2,Manfuha
3,"Manfuha Al-Jadidah (منفوحة الجديدة – ""new Manf..."
4,Al-'Oud
5,Al-Mansorah
6,Al-Margab
7,Salam
8,Jabrah
9,Al-Yamamah


Get the latitude  and longitude. I used https://www.coordinatesfinder.com and google map. The coordinates were saved in Riyadh_districts_coord.xlsx. One district was deleted from the list since it was writen twice

In [125]:
path= 'D:\Github Projects\Applied-Data-Science-Capstone\DataSets\Riyadh_districts_coord.xlsx'
r_cord = pd.read_excel(path)
r_cord

,Neighborhood,latitude,longitude
0,Al-Deerah (old Riyadh),24.630507,46.710825
1,Mi'kal,24.623652,46.714079
2,Manfuha,24.599390,46.727604
3,"Manfuha Al-Jadidah (منفوحة الجديدة – ""new Manf...",24.611303,46.719220
4,Al-'Oud,24.628799,46.726876
5,Al-Mansorah,24.608135,46.764397
6,Al-Margab,24.634916,46.735300
7,Salam,24.621220,46.710104
8,Jabrah,24.625262,46.718733
9,Al-Yamamah,24.595744,46.716083


## Explore and cluster the neighborhoods in Riyadh

In [126]:
address = 'Riyadh, Saudi Arabia'

geolocator = Nominatim(user_agent="ry_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Riyadh are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Riyadh are 24.6319692, 46.7150648.


In [127]:
# create map of Riyadh using latitude and longitude values
map_Riyadh = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for neighborhood, lat, lng in zip(r_cord['Neighborhood'], r_cord['latitude'], r_cord['longitude']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Riyadh)  

map_Riyadh

Define Foursquare Credentials and Version

In [128]:
CLIENT_ID = 'KBTRMD05BNB34QOSMLB0WUO2UOAAEXZT340PHJVJGQXNGRN5' 
CLIENT_SECRET = 'DC1X30JGC43TPYOHVZCUJMIMTUQXYHKSM2H1U3VBEKLM1GWK' 
VERSION = '20200311' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KBTRMD05BNB34QOSMLB0WUO2UOAAEXZT340PHJVJGQXNGRN5
CLIENT_SECRET:DC1X30JGC43TPYOHVZCUJMIMTUQXYHKSM2H1U3VBEKLM1GWK


### Explore Neighborhoods in Riyadh
Create A function to repeat the same process to all the neighborhoods

In [129]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000): # the average neighborhood area is 2km*2km
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
  
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The code to run the above function on each neighborhood and create a new dataframe called *riyadh_venues*.

In [130]:

# the result doesn't includes all the venues in the neighborhood since the radius may not cover all the area of the niehborhood
LIMIT = 100

riyadh_venues = getNearbyVenues(names=r_cord['Neighborhood'],
                                   latitudes=r_cord['latitude'],
                                   longitudes=r_cord['longitude']
                                  )



Al-Deerah (old Riyadh) 
Mi'kal 
Manfuha 
Manfuha Al-Jadidah (منفوحة الجديدة – "new Manfuha") 
Al-'Oud 
Al-Mansorah 
Al-Margab 
Salam 
Jabrah 
Al-Yamamah 
'Otayyigah 
Al-'Olayya 
Al-Sulaymaniyyah 
Al Izdihar 
King Fahd 
Al-Masif
Al-Murooj
Al-Mugharrazat
Al-Wurood
Nemar
Dharat Nemar
Tuwaiq
Hazm
Deerab
Irqah
Al-Khozama
Diplomatic Quarter
Al-Shemaysi
Eleyshah
Syah
Al-Nasriyyah
Umm Sleym
Al-Ma'athar
Al-Olayya
Al-Nakheel
King Saud University main campus
Umm Al-Hamam (East)
Umm Al-Hamam (West)
Al-Ma'athar Al-Shimali ("North Ma'athar")
Al-Rahmaniyya
Al-Muhammadiyya
Al-Ra'id
Al-Ha'ir
Al-Ghannamiyyah
Uraydh
Ad Dar Al Baida
Taybah
Al Mansouriyah
Al-Malaz
Al-Rabwah
Al-Rayyan
Jarir
Al-Murabba'
Al-Masani'
Al-Shifa
Al-Mansuriyya
Al-Marwah
Al-Urayja
Al-Urayja Al-Wusta ("Mid-Urayja")
Al-Urayja (West)
Shubra
Dharat Laban
Hijrat Laban
As-Suwaidi
As-Suwaidi (West)
Al-Badi'a
Dahrat Al-Badi'a
Sultanah
Al-Malga
Al-Sahafa
Hittin
Al-Wadi
Al-Ghadir
Al-Nafil
Imam Muhammad ibn Saud University main campus
Al-Qayra

Check the size of the resulting dataframe

In [131]:
print(riyadh_venues.shape)
riyadh_venues.head(100)

(4504, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Al-Deerah (old Riyadh),24.630507,46.710825,Al Zal Bazaar (سوق الزل التراثي),24.629318,46.712572,Flea Market
1,Al-Deerah (old Riyadh),24.630507,46.710825,Elixir Bunn Coffee Roasters (محمصة اكسير البن),24.630936,46.712279,Coffee Shop
2,Al-Deerah (old Riyadh),24.630507,46.710825,Al Thomairi Old Market (سوق الثميري القديم),24.630015,46.713228,Fabric Shop
3,Al-Deerah (old Riyadh),24.630507,46.710825,Masmak Fortress (قصر المصمك),24.631262,46.713420,Historic Site
4,Al-Deerah (old Riyadh),24.630507,46.710825,المعيقلية للعطور والبخور,24.630153,46.707927,Market
5,Al-Deerah (old Riyadh),24.630507,46.710825,Bread And Coffee (رغيف وكيف),24.630363,46.711909,Coffee Shop
6,Al-Deerah (old Riyadh),24.630507,46.710825,سوق المعيقلية التجاري,24.629998,46.708023,Shopping Mall
7,Al-Deerah (old Riyadh),24.630507,46.710825,محامص ومطاحن السوادي,24.627147,46.709167,Snack Place
8,Al-Deerah (old Riyadh),24.630507,46.710825,Al Thomairi Gold Market (سوق الثميري للذهب),24.630666,46.714202,Jewelry Store
9,Al-Deerah (old Riyadh),24.630507,46.710825,Dunkin’ Donuts,24.628452,46.710463,Donut Shop


In [132]:
len(riyadh_venues['Neighborhood'].unique())

100

Find out how many unique categories can be curated from all the returned venues

In [133]:
print('There are {} uniques categories.'.format(len(riyadh_venues['Venue Category'].unique())))

There are 301 uniques categories.


Convert the Category column into numeric values and separate them into different columns

In [134]:
# one hot encoding
riyadh_onehot = pd.get_dummies(riyadh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
riyadh_onehot['Neighborhood'] = riyadh_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [riyadh_onehot.columns[-1]] + list(riyadh_onehot.columns[:-1])
riyadh_onehot = riyadh_onehot[fixed_columns]

riyadh_onehot.head()

,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bath House,Beach Bar,Bed & Breakfast,Belgian Restaurant,Big Box Store,Boarding House,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Business Center,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal Lock,Candy Store,Car Wash,Carpet Store,Cave,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cupcake Shop,Currency Exchange,Dairy Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dutch Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Pool,Hunting Supply,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kurdish Restaurant,Lake,Lawyer,Lebanese Restaurant,Lighting Store,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Noodle House,North Indian Restaurant,Notary,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,RV Park,Racetrack,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Ski Trail,Slovak Restaurant,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Tennis Court,Thai Restaur

Find the mean value of each category

In [135]:
riyadh_grouped = riyadh_onehot.groupby('Neighborhood').mean().reset_index()

riyadh_grouped.shape

(100, 301)

In [136]:
riyadh_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bath House,Beach Bar,Bed & Breakfast,Belgian Restaurant,Big Box Store,Boarding House,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Business Center,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal Lock,Candy Store,Car Wash,Carpet Store,Cave,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cupcake Shop,Currency Exchange,Dairy Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dutch Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Pool,Hunting Supply,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kurdish Restaurant,Lake,Lawyer,Lebanese Restaurant,Lighting Store,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Noodle House,North Indian Restaurant,Notary,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,RV Park,Racetrack,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Ski Trail,Slovak Restaurant,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Tennis Court,Thai Restaur

Let's check how many venues were returned for each neighorhood

In [137]:
venues_count= riyadh_venues.groupby(["Neighborhood"]).count()['Neighborhood Latitude'].reset_index()
venues_count.columns= ["Neighborhood ","Average venues"]
venues_count

,Neighborhood,Average venues
0,'Otayyigah,27
1,Ad Dar Al Baida,3
2,Ad Difa',1
3,Al Iskan,1
4,Al Izdihar,90
5,Al Mansouriyah,6
6,Al-'Olayya,100
7,Al-'Oud,17
8,Al-Aqiq,76
9,Al-Arid,18


In [138]:
venues_count['Average venues'] = venues_count['Average venues']/venues_count['Average venues'].sum()
venues_count

,Neighborhood,Average venues
0,'Otayyigah,0.005995
1,Ad Dar Al Baida,0.000666
2,Ad Difa',0.000222
3,Al Iskan,0.000222
4,Al Izdihar,0.019982
5,Al Mansouriyah,0.001332
6,Al-'Olayya,0.022202
7,Al-'Oud,0.003774
8,Al-Aqiq,0.016874
9,Al-Arid,0.003996



check if the results contain "Shopping Mall"


In [139]:
'Shopping Mall' in riyadh_venues['Venue Category'].unique()

True

Train using the neighborhoods average number of shopping malls, and the number of venues returned by foursquare API for each neighborhood.  More venues means that the neighborhood tends to be a commercial area

In [140]:
# mall_2 dataframe contains the malls data
mall_2 = riyadh_grouped[['Neighborhood','Shopping Mall']]
mall_2['Venues average']= pd.Series(venues_count['Average venues'])
mall_2

C:\Users\Mar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Neighborhood,Shopping Mall,Venues average
0,'Otayyigah,0.037037,0.005995
1,Ad Dar Al Baida,0.000000,0.000666
2,Ad Difa',0.000000,0.000222
3,Al Iskan,0.000000,0.000222
4,Al Izdihar,0.000000,0.019982
5,Al Mansouriyah,0.000000,0.001332
6,Al-'Olayya,0.000000,0.022202
7,Al-'Oud,0.058824,0.003774
8,Al-Aqiq,0.000000,0.016874
9,Al-Arid,0.000000,0.003996


In [141]:

# set number of clusters
kclusters = 5

riyadh_grouped_clustering = mall_2.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(riyadh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([4, 1, 1, 1, 0, 1, 0, 4, 0, 1, 4, 2, 1, 0, 1, 1, 0, 1, 1, 0, 0, 3,
       0, 1, 1, 1, 1, 1, 3, 1, 0, 0, 3, 0, 0, 1, 1, 3, 1, 1, 1, 2, 0, 1,
       1, 0, 1, 3, 3, 0, 1, 1, 3, 0, 1, 2, 3, 1, 1, 3, 1, 0, 0, 1, 1, 1,
       0, 3, 1, 4, 1, 0, 1, 3, 0, 1, 0, 1, 1, 3, 3, 1, 3, 0, 3, 4, 1, 2,
       0, 0, 2, 0, 3, 1, 1, 1, 0, 1, 1, 1])

mall_df = riyadh_grouped[["Neighborhood","Shopping Mall"]]
mall_df['Cluster']= kmeans.labels_
mall_df

In [142]:
# add cluster data to the mall_2
mall_2['Cluster']= kmeans.labels_


C:\Users\Mar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [143]:
mall_2

,Neighborhood,Shopping Mall,Venues average,Cluster
0,'Otayyigah,0.037037,0.005995,4
1,Ad Dar Al Baida,0.000000,0.000666,1
2,Ad Difa',0.000000,0.000222,1
3,Al Iskan,0.000000,0.000222,1
4,Al Izdihar,0.000000,0.019982,0
5,Al Mansouriyah,0.000000,0.001332,1
6,Al-'Olayya,0.000000,0.022202,0
7,Al-'Oud,0.058824,0.003774,4
8,Al-Aqiq,0.000000,0.016874,0
9,Al-Arid,0.000000,0.003996,1


Add the latitude and longitude to mall_2

In [144]:
#2nd
mall_2 = r_cord.join(mall_2.set_index("Neighborhood"), on='Neighborhood')
mall_2

,Neighborhood,latitude,longitude,Shopping Mall,Venues average,Cluster
0,Al-Deerah (old Riyadh),24.630507,46.710825,0.075949,0.017540,2
1,Mi'kal,24.623652,46.714079,0.096154,0.011545,2
2,Manfuha,24.599390,46.727604,0.043478,0.005107,4
3,"Manfuha Al-Jadidah (منفوحة الجديدة – ""new Manf...",24.611303,46.719220,0.000000,0.003330,1
4,Al-'Oud,24.628799,46.726876,0.058824,0.003774,4
5,Al-Mansorah,24.608135,46.764397,0.000000,0.006661,1
6,Al-Margab,24.634916,46.735300,0.000000,0.002886,1
7,Salam,24.621220,46.710104,0.093750,0.007105,2
8,Jabrah,24.625262,46.718733,0.027778,0.007993,3
9,Al-Yamamah,24.595744,46.716083,0.000000,0.002664,1


Sort the mall dataframe by value of Cluster


In [145]:
#2nd
mall_2.sort_values(["Cluster"], inplace=True)
mall_2.reset_index(drop=True)
mall_2

,Neighborhood,latitude,longitude,Shopping Mall,Venues average,Cluster
91,Al-Qouds,24.753973,46.771963,0.000000,0.020204,0
89,Qortubah (Cordoba),24.816134,46.733935,0.000000,0.022202,0
52,Al-Murabba',24.659914,46.709771,0.000000,0.018872,0
38,"Al-Ma'athar Al-Shimali (""North Ma'athar"")",24.692062,46.665686,0.010000,0.022202,0
36,Umm Al-Hamam (East),24.678706,46.660918,0.000000,0.015098,0
33,Al-Olayya,24.702997,46.678122,0.010000,0.022202,0
32,Al-Ma'athar,24.692162,46.666169,0.010000,0.022202,0
60,Shubra,24.577584,46.671232,0.000000,0.012655,0
26,Diplomatic Quarter,24.677103,46.625145,0.000000,0.015098,0
64,As-Suwaidi (West),24.576600,46.621442,0.000000,0.015764,0


Visualize the mall clusters

In [146]:
#2nd
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mall_2['latitude'], mall_2['longitude'], mall_2['Neighborhood'], mall_2['Cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

There are 5 clusters:
 
<br> 1- Cluster 0 - Low shopping malls, High number of venues </br>
<br> 2- Cluster 1 - Low shoppinng malls, Low number of venues </br>
<br> 3- Cluster 2 - High number of shopping malls, High number of venues </br>
<br> 4- Cluster 3 - Moderate number of shopping malls, Low number of venues </br>
<br> 5- Cluster 4 - Below moderate number of shopping Malls, Moderate number of venues </br>

Create a dataframe that contains cluster 0 data, which are the candidates neigbhorhoods

In [147]:
best_cluster = mall_2[mall_2['Cluster']== 0]
best_cluster

,Neighborhood,latitude,longitude,Shopping Mall,Venues average,Cluster
91,Al-Qouds,24.753973,46.771963,0.00,0.020204,0
89,Qortubah (Cordoba),24.816134,46.733935,0.00,0.022202,0
52,Al-Murabba',24.659914,46.709771,0.00,0.018872,0
38,"Al-Ma'athar Al-Shimali (""North Ma'athar"")",24.692062,46.665686,0.01,0.022202,0
36,Umm Al-Hamam (East),24.678706,46.660918,0.00,0.015098,0
33,Al-Olayya,24.702997,46.678122,0.01,0.022202,0
32,Al-Ma'athar,24.692162,46.666169,0.01,0.022202,0
60,Shubra,24.577584,46.671232,0.00,0.012655,0
26,Diplomatic Quarter,24.677103,46.625145,0.00,0.015098,0
64,As-Suwaidi (West),24.576600,46.621442,0.00,0.015764,0


Visualize cluster 0, which is the candidates neigbhorhoods

In [148]:
#2nd
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(best_cluster['latitude'], best_cluster['longitude'], best_cluster['Neighborhood'], best_cluster['Cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters